LangChain 프롬프트 템플릿
```
LLM 프롬프트를 동적으로 구성하고 재사용할 수 있도록 해주는 도구
    단일입력 : 하나의 변수로 구성된 프롬프트 템플릿
    다중립력 : 둘 이상의 변수를 사용한느 템플릿
    ChatpromptTemplate 역할 기반 프롬프트 : 시스템/사용자 역할별 프롬프트 구성 .from_message()
    PartialPromptTemplate 활용 : 프롬프트 일부를 미리 고정하고 부분포멧칭을 사용(ex 시스템 메시지는 고정,,)
    프롬프트 출력 및 체인 실행 : LCEL
    프롬프트 작성 팁: 주의사항 및 보범사례
```

In [1]:
# 환경설정
%pip install langchain-openai langchain-community python-dotenv openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# 단일 프롬프트 사용
from langchain.prompts import PromptTemplate
# 템플릿 문자열 정의
template_str = (
    "당신은 최고 수준의 마케팅 카피라이터 입니다.\n"
    "아래 제품의 매력적인 홍보문구를 100자 내외로 작성해 주세요\n\n"
    "제품명 : {product_name}"
)
# PromptTemplate 객체 생성
product_prompt = PromptTemplate.from_template(template_str)
# 템플릿에 값 채우기
formatted_prompt = product_prompt.format(product_name="이클립스")
print(formatted_prompt)

당신은 최고 수준의 마케팅 카피라이터 입니다.
아래 제품의 매력적인 홍보문구를 100자 내외로 작성해 주세요

제품명 : 이클립스


In [5]:
# 프롬프트 | llm -> invoke
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
# Runnable 객체 생성 LCEL
chain = product_prompt | llm
result = chain.invoke({"product_name": "이클립스"})
print(result.content)


"이클립스, 당신의 일상에 빛나는 변화를! 매일의 순간을 특별하게 만들어주는 혁신적인 경험을 만나보세요."


In [6]:
# 다중 입력
# 다중입력 템플릿 문자열 정의
multi_template_str = (
    "아래는 뉴스 기사 제목과 키워드 입니다다.\n"
    "이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요요\n\n"
    "제목 : {title}\n"
    "키워드 : {keywords}"
)
# 프롬프트 템플릿 작성
summary_prompt = PromptTemplate(template=multi_template_str,input_variables=["title", "keywords"])
# 포멧팅을 통해 프롬프트 값 확인
sample_title = "인공지능 기술의 발전과 미래"
sample_keywords = "머신러닝, 딥러닝, LLM, 산업 혁신"
formatted_summary_prompt = summary_prompt.format(title=sample_title, keywords=sample_keywords)
print(formatted_summary_prompt)

아래는 뉴스 기사 제목과 키워드 입니다다.
이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요요

제목 : 인공지능 기술의 발전과 미래
키워드 : 머신러닝, 딥러닝, LLM, 산업 혁신


In [7]:
# LCEL 출력
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
result_summary = (summary_prompt | llm | parser).invoke({
    "title": sample_title,
    "keywords": sample_keywords
})
result_summary

'인공지능 기술의 발전은 머신러닝과 딥러닝의 혁신을 통해 가속화되고 있으며, 특히 대규모 언어 모델(LLM)의 등장으로 다양한 산업 분야에서의 혁신이 이루어지고 있습니다. 이러한 기술들은 데이터 분석, 자동화, 고객 서비스 개선 등에서 중요한 역할을 하며, 기업들이 경쟁력을 유지하고 새로운 비즈니스 기회를 창출하는 데 기여하고 있습니다. 앞으로 인공지능 기술은 더욱 발전하여 우리의 삶과 산업 구조에 큰 변화를 가져올 것으로 기대됩니다.'

In [8]:
# ChatPromptTemplate과 역할 기반 프롬프트
# 시스템/사용자/어시스턴스 역할(role)
# 시스템 메시지 ㅣ 모델의 동작을 지시
# 사용자 메시지 ㅣ 실제 사용자의 입력
# 어시스턴트 메시지 : 이전 모델이 응답한 내용이 있으면 대화맥락유지를 위해 활용 

In [11]:
from langchain.prompts import ChatPromptTemplate
system_message = ("당신은 python 분야의 뛰어난 전문가이자 조언자 입니다"
                  "자용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 답변해 주세요")
user_message = "{question}"
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", user_message)
])
# 템플릿을 이용해서 문장을 완성해 보세요
sample_question = "파이썬의 클래스에 대해서 설명해주세요"
formatted_chat_prompt = chat_prompt.format(question=sample_question)
print(formatted_chat_prompt)
# 파이프라인을 이용해서 llm 호출 및 출력
answer = (chat_prompt | llm | parser).invoke({"question": sample_question})
print(answer)



System: 당신은 python 분야의 뛰어난 전문가이자 조언자 입니다자용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 답변해 주세요
Human: 파이썬의 클래스에 대해서 설명해주세요
파이썬의 클래스는 객체 지향 프로그래밍(OOP)의 기본 개념 중 하나로, 객체를 생성하기 위한 청사진(또는 템플릿)입니다. 클래스를 사용하면 관련된 데이터와 기능을 하나의 단위로 묶을 수 있습니다. 이를 통해 코드의 재사용성과 유지보수성을 높일 수 있습니다.

### 클래스의 기본 구조

클래스를 정의하려면 `class` 키워드를 사용합니다. 다음은 간단한 클래스의 예입니다:

```python
class Dog:
    # 생성자 메서드
    def __init__(self, name, age):
        self.name = name  # 인스턴스 변수
        self.age = age    # 인스턴스 변수

    # 메서드
    def bark(self):
        return f"{self.name} says Woof!"

# 클래스의 인스턴스 생성
my_dog = Dog("Buddy", 3)

# 메서드 호출
print(my_dog.bark())  # 출력: Buddy says Woof!
```

### 주요 개념

1. **생성자 (`__init__` 메서드)**:
   - 클래스의 인스턴스가 생성될 때 호출되는 특별한 메서드입니다.
   - 인스턴스 변수를 초기화하는 데 사용됩니다.

2. **인스턴스 변수**:
   - 각 인스턴스(객체)마다 고유한 데이터를 저장하는 변수입니다.
   - `self` 키워드를 사용하여 인스턴스 변수를 정의합니다.

3. **메서드**:
   - 클래스 내에서 정의된 함수로, 클래스의 인스턴스에서 호출할 수 있습니다.
   - 메서드는 일반적으로 인스턴스 변수에 접근하거나 조작하는 데 사용됩니다.

4. **인스턴스**:
   - 클래스를 기반으로 생성된 객체입니다. 각 인스턴스는 클래스의 속성과 메서

In [17]:
# ParitalPromptTemplate : 템플릿의 일부를 부분적으로 채운 새로운 탬플릿
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
role_system_template = '당신은 {role} 분야의 전문 지식인 입니다. 가능한 자세히 답변해 주세요'
system_prompt = SystemMessagePromptTemplate.from_template(role_system_template)
user_prompt = HumanMessagePromptTemplate.from_template("{question}")

# chatprompttemplate를 생성
base_chat_prompt = ChatPromptTemplate.from_messages([system_prompt,user_prompt])
partial_chat_prompt = base_chat_prompt.partial(role="주식 투자")

In [18]:
# partial로 생성된 프롬프트에 질문만 채워 프롬프트 구성
sample_question = "현재 2025년 5월 시장 상황에서 삼성전자 주식 전망은?"
message = partial_chat_prompt.format_messages(question=sample_question)
print(message)

[SystemMessage(content='당신은 주식 투자 분야의 전문 지식인 입니다. 가능한 자세히 답변해 주세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='현재 2025년 5월 시장 상황에서 삼성전자 주식 전망은?', additional_kwargs={}, response_metadata={})]


In [19]:
# LCEL - invoke
answer = (partial_chat_prompt | llm | parser).invoke({"question": sample_question})
print(answer)

2025년 5월의 삼성전자 주식 전망을 평가하기 위해서는 여러 가지 요소를 고려해야 합니다. 다음은 삼성전자의 주식 전망에 영향을 미칠 수 있는 주요 요인들입니다.

1. **반도체 시장 동향**: 삼성전자는 세계 최대의 반도체 제조업체 중 하나입니다. 반도체 수요는 AI, IoT, 5G 등 다양한 기술 발전에 따라 증가하고 있습니다. 2025년에는 이러한 기술들이 더욱 발전할 것으로 예상되며, 이에 따라 반도체 수요가 증가할 가능성이 높습니다. 그러나 반도체 시장은 경기 사이클에 민감하므로, 글로벌 경제 상황에 따라 변동성이 클 수 있습니다.

2. **스마트폰 및 가전제품 판매**: 삼성전자는 스마트폰과 가전제품에서도 강력한 입지를 가지고 있습니다. 2025년에는 새로운 모델과 혁신적인 기술이 출시될 가능성이 있으며, 이는 소비자 수요에 긍정적인 영향을 미칠 수 있습니다. 특히, 폴더블 스마트폰과 같은 혁신적인 제품이 시장에서 성공을 거두면 주가에 긍정적인 영향을 미칠 수 있습니다.

3. **글로벌 경제 상황**: 글로벌 경제의 성장률, 금리, 인플레이션 등은 삼성전자의 주가에 큰 영향을 미칠 수 있습니다. 경제가 안정적이고 성장세를 보인다면 소비자 지출이 증가하고, 이는 삼성전자의 매출 증가로 이어질 수 있습니다.

4. **경쟁 상황**: 삼성전자는 애플, 화웨이, TSMC 등과 같은 강력한 경쟁자들과 경쟁하고 있습니다. 경쟁사의 기술 발전이나 시장 점유율 변화는 삼성전자의 주가에 영향을 미칠 수 있습니다.

5. **정치적 및 규제 환경**: 반도체 산업은 국가 간의 정치적 긴장이나 규제 변화에 민감합니다. 예를 들어, 미국과 중국 간의 무역 갈등이나 반도체 관련 규제는 삼성전자의 사업에 영향을 미칠 수 있습니다.

6. **ESG(환경, 사회, 지배구조) 경영**: 최근 투자자들은 ESG 요소를 중요하게 고려하고 있습니다. 삼성전자가 지속 가능한 경영을 강화하고 사회적 책임을 다하는 모습을 보인다면, 이는 긍정적인 투자 심리를 유도할 수 있습니다.